In [2]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score



In [7]:
#ATMS 523 HW 5 

#1) split the data into a 70-30 split for training and test data 

# First, load the data
df = pd.read_csv('radar_parameters.csv')


 

In [8]:
# Create feature/predictor matrix. Split them 70-30 training-test 

X = df.iloc[:,1:7]

Y = df.iloc[:,7]

Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3,
                                                random_state=42) 


In [6]:
#Init linear regression model 
model = LinearRegression(fit_intercept=True) 

In [7]:
# 2. Using the split created in (1), train a multiple linear regression dataset using the training dataset, and validate it using the testing dataset.  
# Compare the $R^2$ and root mean square errors of model on the training and testing sets to a baseline prediction of rain rate using the formula $Z = 200 R^{1.6}$.

mdl = model.fit(Xtrain,ytrain)





In [8]:
#Validate model using testing set - two metrics (R^2, RMSE)

ypred_test = mdl.predict(Xtest)
ypred_train = mdl.predict(Xtrain)

r2_validation = r2_score(ytest, ypred_test)
rmse_validation = mean_squared_error(ytest,ypred_test,squared=False)
print(f'R^2 testing data set: {r2_validation}')
print(f'RMSE testing data set: {rmse_validation}')

r2_validation_training_data = r2_score(ytrain, ypred_train)
rmse_validation_training_data = mean_squared_error(ytrain,ypred_train,squared=False)
print(f'R^2 training data set: {r2_validation_training_data}')
print(f'RMSE training data set: {rmse_validation_training_data}')



R^2 testing data set: 0.9890992951689396
RMSE testing data set: 0.935812474208696
R^2 training data set: 0.9879085512445995
RMSE training data set: 0.9229401590287888


In [9]:
## Use formula Zh = 200*R^1.6 as a baseline comparison for training and testing datasets 
# R = Zh^(1/1.6)/200

Zh_train_dBZ = Xtrain.iloc[:,0]
Zh_test_dBZ = Xtest.iloc[:,0]

Zh_train = 10 ** (Zh_train_dBZ/10)
Zh_test= 10 ** (Zh_test_dBZ/10)

ypred_test_baseline = (Zh_test/200) ** (1/1.6)
ypred_train_baseline = (Zh_train/200) ** (1/1.6)

r2_validation_test_baseline = r2_score(ytest, ypred_test_baseline)
rmse_validation_test_baseline = mean_squared_error(ytest,ypred_test_baseline,squared=False)
print(f'R^2 baseline testing data set: {r2_validation_test_baseline}')
print(f'RMSE baseline testing data set: {rmse_validation_test_baseline}')

r2_validation_training_data_baseline = r2_score(ytrain, ypred_train_baseline)
rmse_validation_training_data_baseline = mean_squared_error(ytrain,ypred_train_baseline,squared=False)
print(f'R^2 baseline training data set: {r2_validation_training_data_baseline}')
print(f'RMSE baseline training data set: {rmse_validation_training_data_baseline}')

R^2 baseline testing data set: 0.35664291868109677
RMSE baseline testing data set: 7.189316160047872
R^2 baseline training data set: 0.27555056242697507
RMSE baseline training data set: 7.143950117300888


3. Repeat 1 doing a grid search over polynomial orders, using a grid search over orders 0-21, and use cross-validation of 7 folds.  For the best polynomial model in terms of $R^2$, does it outperform the baseline and the linear regression model in terms of $R^2$ and root mean square error?

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

from sklearn.model_selection import GridSearchCV


#ONLY PICKED 10 PARAMS B/C OF COMPUTATIONAL ISSUES 
param_grid = {'polynomialfeatures__degree': np.arange(10)}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)



In [11]:
grid.fit(Xtrain, ytrain)

GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'polynomialfeatures__degree': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [12]:
model = grid.best_estimator_

ypred_train_poly = model.predict(Xtrain)
ypred_test_poly = model.predict(Xtest)

In [13]:
r2_validation_test_baseline = r2_score(ytest, ypred_test_poly)
rmse_validation_test_baseline = mean_squared_error(ytest,ypred_test_poly,squared=False)
print(f'R^2 polynominal model testing data set: {r2_validation_test_baseline}')
print(f'RMSE polynomial model testing data set: {rmse_validation_test_baseline}')

r2_validation_training_data_baseline = r2_score(ytrain, ypred_train_poly)
rmse_validation_training_data_baseline = mean_squared_error(ytrain,ypred_train_poly,squared=False)
print(f'R^2 polynomial model training data set: {r2_validation_training_data_baseline}')
print(f'RMSE polynomial model training data set: {rmse_validation_training_data_baseline}')

R^2 polynominal model testing data set: 0.9995805761884335
RMSE polynomial model testing data set: 0.18356424030927726
R^2 polynomial model training data set: 0.9996032995956293
RMSE polynomial model training data set: 0.16717283026815857


# Based on the results above (NOTE: only did 10 order polynomial because my computer struggled mightily with anything above that)

# The polynomial model best fit seemed to outperform the multivariable linear regression. 

4. Repeat 1 with a Random Forest Regressor, and perform a grid_search on the following parameters:
   
   ```python
   {'bootstrap': [True, False],  
   'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  
   'max_features': ['auto', 'sqrt'],  
   'min_samples_leaf': [1, 2, 4],  
   'min_samples_split': [2, 5, 10],  
   'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
   ```
  Can you beat the baseline, or the linear regression, or best polynomial model with the best optimized Random Forest Regressor in terms of $R^2$ and root mean square error?




In [15]:
from sklearn.ensemble import RandomForestRegressor


# Define the Random Forest Regressor and grid search parameters
rf = RandomForestRegressor(random_state=42)
param_grid = {
'bootstrap': [True],
'max_depth': [50, 100],
'max_features': ['auto'],
'min_samples_leaf': [4],
'min_samples_split': [10],
'n_estimators': [1400,1800]} 

# Set up the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid)



In [16]:
# Perform the grid search
grid_search.fit(Xtrain, ytrain)

/Users/sunnysharma/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/sunnysharma/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/sunnysharma/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this param

GridSearchCV(estimator=RandomForestRegressor(random_state=42),
             param_grid={'bootstrap': [True], 'max_depth': [50, 100],
                         'max_features': ['auto'], 'min_samples_leaf': [4],
                         'min_samples_split': [10],
                         'n_estimators': [1400, 1800]})

In [21]:

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
predictions = best_model.predict(Xtest)
mse = mean_squared_error(ytest, predictions)

print("Mean Squared Error of the best model: ", mse)

r2 = r2_score(ytest, predictions)
print(f'R2 of best model = {r2}')


Best parameters found:  {'bootstrap': True, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1800}
Mean Squared Error of the best model:  1.2722953889351536
R2 of best model = 0.9841632933098037
